# Installing Required Libraries

In [ ]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

# Importing Libraries and Warning Control

In [ ]:
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

# Warning control
import warnings
warnings.filterwarnings('ignore')

# Defining the LLM (Language Model) & Serper Tool:

In [ ]:
llm = ChatGroq(temperature=0,
             model_name="llama3-8b-8192",
             api_key='You should write your API KEY here.')

import os
# Serper API anahtarını doğrudan çevresel değişken olarak ayarlayın
os.environ["SERPER_API_KEY"] = "You should write your SERPER API KEY here."


# Creating Tools

**crewAI Tools**

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
# semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md') -> OpenAI Tool olduğu için olmuyor. Normalde bu lazım..

**Uncomment and run the cell below if you wish to view fake_resume.md in the notebook.**

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

# Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    llm = llm,
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    llm = llm,
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,  #  ->  tools = [scrape_tool, search_tool,
             read_resume],              #  ->  read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    llm = llm,
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,  #  ->  tools = [scrape_tool, search_tool,
             read_resume],              #  ->  read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    llm = llm,
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,  #  ->  tools = [scrape_tool, search_tool,
             read_resume],              #  ->  read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

# Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


# Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

# Running the Crew

*   Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
- Developed a full-stack e-commerce application using React and Node.js.
- Implemented user authentication, product catalog, and payment processing features.
- Deployed the application on AWS, utilizing EC2, S3, and RDS services.

**Real-Time Chat Application**
- Built a real-time chat application using WebSocket and Node.js.
- Implemented features like private messaging, group chats, and notifications.
- Ensured data security and privacy through end-to-end encryption.

---

### Professional Memberships
- **Member, Association for Computing Machinery (ACM)**
- **Member, IEEE Computer Society**

---

### Languages
- **English:** Native
- **Spanish:** Fluent
- **German:** Conversational

---

### Interests
- Open-source contributions
- AI and Machine Learning
- Running marathons
- Traveling and exploring new cultures


Thought: I need to extract relevant information from Noah's write-up to create a standout resume.

Action: Read a file's con

# Display the results of your execution as markdown.

**Dislplay the generated tailored_resume.md file.**

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

The updated resume highlights the most relevant areas of John Doe's skills and experiences, including cloud-based applications development, microservices architecture, Agile development processes, front-end optimization techniques, and certifications in Kubernetes and AWS.

**Dislplay the generated interview_materials.md file.**

In [ ]:
display(Markdown("./interview_materials.md"))

**Potential Interview Questions and Talking Points**

Based on the tailored resume and job requirements, here are some potential interview questions and talking points that the candidate should prepare for:

**Leadership Experience**

* Can you describe your experience leading a team of engineers? How did you motivate and manage them?
* How do you prioritize and delegate tasks to your team members?
* Can you give an example of a project you led and the results you achieved?

**Technical Skills**

* Can you explain your experience with cloud-based applications and microservices architecture?
* How do you stay up-to-date with the latest technologies and trends in software engineering?
* Can you describe your experience with Agile development methodologies and how you implement them in your work?

**Collaboration and Communication**

* Can you describe your experience working with cross-functional teams and stakeholders?
* How do you communicate technical information to non-technical stakeholders?
* Can you give an example of a time when you had to resolve a conflict or issue with a team member or stakeholder?

**Certifications and Education**

* Can you describe your experience with Kubernetes and AWS?
* How do you stay current with the latest certifications and training in software engineering?
* Can you describe your educational background and how it has prepared you for your career in software engineering?

**Projects and Achievements**

* Can you describe your experience with developing full-stack e-commerce applications and real-time chat applications?
* How do you approach problem-solving and debugging in your code?
* Can you give an example of a project you worked on and the results you achieved?

**Interests and Hobbies**

* Can you describe your interests and hobbies outside of work?
* How do you balance your work and personal life?
* Can you give an example of a time when you had to adapt to a new situation or challenge?

These questions and talking points are designed to help the candidate showcase their skills, experience, and achievements in software engineering leadership, as well as their ability to collaborate and communicate effectively with others.